In [1]:
from io_utils import *
import pandas as pd
from astropy.table import Table, vstack
from random import random

In [2]:
dataset = "ZTF"
#object_ids = data_ob.get_all_object_ids()
#kilonova_ids = object_ids
bands = ['g', 'r']
color_band_dict = {'g':'C2', 'r':'C3'}

In [3]:
m_numbers = [101, 102, 103, 112, 113, 114, 141, 143, 145, 150, 151, 160, 161, 162, 163, 164, 170, 180, 181, 183, 190]

In [4]:
print(len(m_numbers))

21


In [5]:
#data_ob = load_ztf_mixed(m_numbers=m_numbers)

In [6]:
base_path = '/media/biswajit/drive/Kilonova_datasets/ZTF_20190512/ZTF_MSIP_MODEL'

In [7]:
def create_dataset(m_numbers, start_number, end_number, total_num_kn, total_num_non_kn):
    num_kn_models = np.sum(np.isin([145,150],m_numbers))
    if total_num_non_kn is not None:
        av_num_of_non_kn_models = 0 
        if len(m_numbers)!=num_kn_models:
            av_num_of_non_kn_models = total_num_non_kn/(len(m_numbers)-num_kn_models)
    if total_num_kn is not None:  
        av_num_of_kn_models = 0 
        if num_kn_models!=0:
            av_num_of_kn_models =  total_num_kn/num_kn_models
    data_ob = None
    for model_num in m_numbers:
        model_ob = None
        for file_num in range(start_number, end_number):
            
            if (model_num!=145)&(model_num!=150):
                if file_num<10:
                    file_path = base_path+str(model_num)[-2:]+'/ZTF_MSIP_NONIaMODEL0-000'+str(file_num)+'_PHOT.FITS'
                    current_ob = load_ztf_data(phot_path=file_path)
                else: 
                    file_path = base_path+str(model_num)[-2:]+'/ZTF_MSIP_NONIaMODEL0-00'+str(file_num)+'_PHOT.FITS'
                    current_ob = load_ztf_data(phot_path=file_path)
            else:
                if file_num<10:
                    file_path = base_path+str(model_num)[-2:]+'/ZTF_MSIP_NONIa-000'+str(file_num)+'_PHOT.FITS'
                    current_ob = load_ztf_data(phot_path=file_path)
                else:
                    file_path = base_path+str(model_num)[-2:]+'/ZTF_MSIP_NONIa-00'+str(file_num)+'_PHOT.FITS'
                    current_ob = load_ztf_data(phot_path=file_path)
                    
            print(len(current_ob.df_metadata))
            if model_ob is None: 
                    model_ob = current_ob
            else:
                model_ob.df_data= vstack([model_ob.df_data,current_ob.df_data])
                model_ob.df_metadata = vstack([model_ob.df_metadata, current_ob.df_metadata], join_type='inner')
        if data_ob is not None:
            id_clash = np.isin(model_ob.df_metadata['SNID'], data_ob.df_metadata['SNID'])
            model_ob.df_metadata = model_ob.df_metadata[~id_clash]

        np.random.permutation(model_ob.df_metadata)
        if (model_num==151)|(model_num==150):
            if total_num_kn is not None:
                model_ob.df_metadata = model_ob.df_metadata[0:int(av_num_of_kn_models)]
            print()
        else:
            if total_num_non_kn is not None:
                model_ob.df_metadata = model_ob.df_metadata[0:int(av_num_of_non_kn_models)]
        model_ob.df_data = model_ob.df_data[np.isin(model_ob.df_data['SNID'],model_ob.df_metadata['SNID'])]
        print(len(model_ob.df_metadata))
        print(len(np.unique(model_ob.df_metadata['SNID'])))
        print(len(np.unique(model_ob.df_data['SNID'])))
        #print(model_ob.df_data['SNID'])
        print("--------------------------")
        if data_ob is None:
            data_ob = model_ob
        else: 
            data_ob.df_metadata = vstack([data_ob.df_metadata, model_ob.df_metadata], join_type='inner', metadata_conflicts='error')
            data_ob.df_data = vstack([data_ob.df_data, model_ob.df_data], join_type='inner', metadata_conflicts='error')
            
            print(len(data_ob.df_metadata))
            print(len(np.unique(data_ob.df_metadata['SNID'])))
            print(len(np.unique(data_ob.df_data['SNID'])))
    return data_ob


In [8]:
test_ob = create_dataset(m_numbers, 1,20,2000,19000)

319
314
259
311
297
278
296
294
288
281
341
298
297
299
299
309
305
316
333
1000
1000
1000
--------------------------
269
276
243
261
265
251
268
244
260
253
226
271
259
265
243
242
246
255
264
1000
1000
1000
--------------------------
2000
2000
2000
261
275
273
257
246
245
247
264
231
268
241
289
273
245
244
255
273
268
271
1000
1000
1000
--------------------------
3000
3000
3000
282
224
267
251
272
275
227
248
251
276
289
257
248
306
261
234
271
277
254
1000
1000
1000
--------------------------
4000
4000
4000
242
233
242
250
266
247
264
254
285
273
249
281
247
275
268
273
281
279
284
1000
1000
1000
--------------------------
5000
5000
5000
266
274
248
266
279
260
258
282
287
253
292
279
286
313
291
284
292
294
269
1000
1000
1000
--------------------------
6000
6000
6000
270
234
247
246
256
262
274
275
243
236
250
266
242
266
251
238
262
250
278
1000
1000
1000
--------------------------
7000
7000
7000
267
260
268
283
277
270
253
263
261
248
289
244
276
267
235
248
274
274
257
1000
100

In [9]:
test_ob.df_metadata

SUBSURVEY,SNID,IAUC,FAKE,RA,PIXSIZE,NXPIX,NYPIX,CCDNUM,SNTYPE,NOBS,PTROBS_MIN,PTROBS_MAX,MWEBV,MWEBV_ERR,REDSHIFT_HELIO,REDSHIFT_HELIO_ERR,REDSHIFT_FINAL,REDSHIFT_FINAL_ERR,VPEC,VPEC_ERR,HOSTGAL_OBJID,HOSTGAL_PHOTOZ,HOSTGAL_PHOTOZ_ERR,HOSTGAL_SPECZ,HOSTGAL_SPECZ_ERR,HOSTGAL_SNSEP,HOSTGAL_LOGMASS,HOSTGAL_LOGMASS_ERR,HOSTGAL_MAG_g,HOSTGAL_MAG_r,HOSTGAL_SB_FLUXCAL_g,HOSTGAL_SB_FLUXCAL_r,PEAKMJD,SEARCH_TYPE,SIM_MODEL_NAME,SIM_MODEL_INDEX,SIM_TYPE_INDEX,SIM_TYPE_NAME,SIM_LIBID,SIM_NGEN_LIBID,SIM_NOBS_UNDEFINED,SIM_SEARCHEFF_MASK,SIM_REDSHIFT_HELIO,SIM_REDSHIFT_CMB,SIM_REDSHIFT_HOST,SIM_VPEC,SIM_DLMU,SIM_LENSDMU,SIM_RA,SIM_MWEBV,SIM_PEAKMJD,SIM_MAGSMEAR_COH,SIM_AV,SIM_RV,SIM_PEAKMAG_g,SIM_PEAKMAG_r,SIM_EXPOSURE_g,SIM_EXPOSURE_r,SIM_SUBSAMPLE_INDEX
bytes40,int32,bytes16,int16,float64,float32,int16,int16,int16,int32,int32,int32,int32,float32,float32,float32,float32,float32,float32,float32,float32,int64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int32,bytes32,int16,int16,bytes8,int32,int32,int32,int32,float32,float32,float32,float32,float32,float32,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16
,72836215,NULL,2,1.1815536951468222,1.0,-9,-9,0,101,34,1,34,0.026526676,0.0013263338,-9.0,-9.0,-9.0,-9.0,0.0,300.0,-9,-9.0,-9.0,0.0,0.0,-9.0,-9.0,-9.0,999.0,999.0,-9.0,-9.0,57704.766,-9,SALT2.WFIRST-H17,6,1,UNKNOWN,254,1,0,1,0.13947846,0.138431,0.13947846,-93.788124,39.077847,-0.009440493,1.1815537214279175,0.02752111,57704.277,0.061459836,-9.0,-9.0,20.298021,20.218906,1.0,1.0,-9
,60291778,NULL,2,5.013103625449706,1.0,-9,-9,0,101,5,36,40,0.026615221,0.001330761,-9.0,-9.0,-9.0,-9.0,0.0,300.0,-9,-9.0,-9.0,0.0,0.0,-9.0,-9.0,-9.0,999.0,999.0,-9.0,-9.0,57531.492,-9,SALT2.WFIRST-H17,6,1,UNKNOWN,284,1,0,1,0.17979395,0.1784641,0.17979395,-15.915529,39.68362,-0.009008044,5.013103485107422,0.026272193,57531.164,-0.099815406,-9.0,-9.0,20.253166,20.227411,1.0,1.0,-9
,112270926,NULL,2,0.15200142268788697,1.0,-9,-9,0,101,54,42,95,0.02887419,0.0014437096,-9.0,-9.0,-9.0,-9.0,0.0,300.0,-9,-9.0,-9.0,0.0,0.0,-9.0,-9.0,-9.0,999.0,999.0,-9.0,-9.0,57578.676,-9,SALT2.WFIRST-H17,6,1,UNKNOWN,295,1,0,1,0.20317169,0.20141903,0.20317169,93.195366,39.976597,0.0010082603,0.15200142562389374,0.028675895,57576.957,0.039121453,-9.0,-9.0,20.537012,20.496603,1.0,1.0,-9
,19447765,NULL,2,1.5116340284754186,1.0,-9,-9,0,101,43,97,139,0.029253114,0.0014626557,-9.0,-9.0,-9.0,-9.0,0.0,300.0,-9,-9.0,-9.0,0.0,0.0,-9.0,-9.0,-9.0,999.0,999.0,-9.0,-9.0,57591.625,-9,SALT2.WFIRST-H17,6,1,UNKNOWN,306,1,0,1,0.07950008,0.076921284,0.07950008,387.7436,37.71647,-0.00034464168,1.5116339921951294,0.02788574,57592.117,0.09290558,-9.0,-9.0,18.451132,18.540688,1.0,1.0,-9
,124363533,NULL,2,2.8645603811450875,1.0,-9,-9,0,101,23,141,163,0.02636083,0.0013180415,-9.0,-9.0,-9.0,-9.0,0.0,300.0,-9,-9.0,-9.0,0.0,0.0,-9.0,-9.0,-9.0,999.0,999.0,-9.0,-9.0,57458.082,-9,SALT2.WFIRST-H17,6,1,UNKNOWN,317,1,0,1,0.14827746,0.14720915,0.14827746,-87.79753,39.223415,-0.022519,2.86456036567688,0.026454661,57458.035,0.032284915,-9.0,-9.0,20.053347,20.10028,1.0,1.0,-9
,120475786,NULL,2,5.446362937234302,1.0,-9,-9,0,101,42,165,206,0.022678046,0.0011339023,-9.0,-9.0,-9.0,-9.0,0.0,300.0,-9,-9.0,-9.0,0.0,0.0,-9.0,-9.0,-9.0,999.0,999.0,-9.0,-9.0,57629.938,-9,SALT2.WFIRST-H17,6,1,UNKNOWN,338,1,0,1,0.29929033,0.29828295,0.29929033,-153.81381,40.942772,-0.0234933,5.4463629722595215,0.021109732,57632.477,0.0077110054,-9.0,-9.0,21.287857,21.209156,1.0,1.0,-9
,101712602,NULL,2,0.9519917603624842,1.0,-9,-9,0,101,53,208,260,0.029481145,0.0014740572,-9.0,-9.0,-9.0,-9.0,0.0,300.0,-9,-9.0,-9.0,0.0,0.0,-9.0,-9.0,-9.0,999.0,999.0,-9.0,-9.0,57679.04,-9,SALT2.WFIRST-H17,6,1,UNKNOWN,352,1,0,1,0.11761785,0.11576325,0.11761785,155.70459,38.659367,0.0020025002,0.9519917368888855,0.028507305,57679.4,0.1567895,-9.0,-9.0,19.27869,19.40735,1.0,1.0,-9
,11337994,NULL,2,3.2937010649608274,1.0,-9,-9,0,101,25,262,286,0.02496931,0.0012484655,-9.0,-9.0,-9.0,-9.0,0.0,300

In [10]:
print(len(np.unique(test_ob.df_metadata)))

20788


In [11]:
len(np.unique(test_ob.df_metadata['SNID']))

20788

In [12]:
print(np.sort(np.unique(test_ob.df_data['SNID'])))

   SNID  
---------
    10871
    10872
    11422
    13390
    13997
    15596
    17873
    18008
    23181
    48431
      ...
137018130
137028008
137029914
137034960
137043138
137044385
137049400
137051059
137054088
137071432
137078995
Length = 20788 rows


In [13]:
test_ob.df_metadata

SUBSURVEY,SNID,IAUC,FAKE,RA,PIXSIZE,NXPIX,NYPIX,CCDNUM,SNTYPE,NOBS,PTROBS_MIN,PTROBS_MAX,MWEBV,MWEBV_ERR,REDSHIFT_HELIO,REDSHIFT_HELIO_ERR,REDSHIFT_FINAL,REDSHIFT_FINAL_ERR,VPEC,VPEC_ERR,HOSTGAL_OBJID,HOSTGAL_PHOTOZ,HOSTGAL_PHOTOZ_ERR,HOSTGAL_SPECZ,HOSTGAL_SPECZ_ERR,HOSTGAL_SNSEP,HOSTGAL_LOGMASS,HOSTGAL_LOGMASS_ERR,HOSTGAL_MAG_g,HOSTGAL_MAG_r,HOSTGAL_SB_FLUXCAL_g,HOSTGAL_SB_FLUXCAL_r,PEAKMJD,SEARCH_TYPE,SIM_MODEL_NAME,SIM_MODEL_INDEX,SIM_TYPE_INDEX,SIM_TYPE_NAME,SIM_LIBID,SIM_NGEN_LIBID,SIM_NOBS_UNDEFINED,SIM_SEARCHEFF_MASK,SIM_REDSHIFT_HELIO,SIM_REDSHIFT_CMB,SIM_REDSHIFT_HOST,SIM_VPEC,SIM_DLMU,SIM_LENSDMU,SIM_RA,SIM_MWEBV,SIM_PEAKMJD,SIM_MAGSMEAR_COH,SIM_AV,SIM_RV,SIM_PEAKMAG_g,SIM_PEAKMAG_r,SIM_EXPOSURE_g,SIM_EXPOSURE_r,SIM_SUBSAMPLE_INDEX
bytes40,int32,bytes16,int16,float64,float32,int16,int16,int16,int32,int32,int32,int32,float32,float32,float32,float32,float32,float32,float32,float32,int64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int32,bytes32,int16,int16,bytes8,int32,int32,int32,int32,float32,float32,float32,float32,float32,float32,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16
,72836215,NULL,2,1.1815536951468222,1.0,-9,-9,0,101,34,1,34,0.026526676,0.0013263338,-9.0,-9.0,-9.0,-9.0,0.0,300.0,-9,-9.0,-9.0,0.0,0.0,-9.0,-9.0,-9.0,999.0,999.0,-9.0,-9.0,57704.766,-9,SALT2.WFIRST-H17,6,1,UNKNOWN,254,1,0,1,0.13947846,0.138431,0.13947846,-93.788124,39.077847,-0.009440493,1.1815537214279175,0.02752111,57704.277,0.061459836,-9.0,-9.0,20.298021,20.218906,1.0,1.0,-9
,60291778,NULL,2,5.013103625449706,1.0,-9,-9,0,101,5,36,40,0.026615221,0.001330761,-9.0,-9.0,-9.0,-9.0,0.0,300.0,-9,-9.0,-9.0,0.0,0.0,-9.0,-9.0,-9.0,999.0,999.0,-9.0,-9.0,57531.492,-9,SALT2.WFIRST-H17,6,1,UNKNOWN,284,1,0,1,0.17979395,0.1784641,0.17979395,-15.915529,39.68362,-0.009008044,5.013103485107422,0.026272193,57531.164,-0.099815406,-9.0,-9.0,20.253166,20.227411,1.0,1.0,-9
,112270926,NULL,2,0.15200142268788697,1.0,-9,-9,0,101,54,42,95,0.02887419,0.0014437096,-9.0,-9.0,-9.0,-9.0,0.0,300.0,-9,-9.0,-9.0,0.0,0.0,-9.0,-9.0,-9.0,999.0,999.0,-9.0,-9.0,57578.676,-9,SALT2.WFIRST-H17,6,1,UNKNOWN,295,1,0,1,0.20317169,0.20141903,0.20317169,93.195366,39.976597,0.0010082603,0.15200142562389374,0.028675895,57576.957,0.039121453,-9.0,-9.0,20.537012,20.496603,1.0,1.0,-9
,19447765,NULL,2,1.5116340284754186,1.0,-9,-9,0,101,43,97,139,0.029253114,0.0014626557,-9.0,-9.0,-9.0,-9.0,0.0,300.0,-9,-9.0,-9.0,0.0,0.0,-9.0,-9.0,-9.0,999.0,999.0,-9.0,-9.0,57591.625,-9,SALT2.WFIRST-H17,6,1,UNKNOWN,306,1,0,1,0.07950008,0.076921284,0.07950008,387.7436,37.71647,-0.00034464168,1.5116339921951294,0.02788574,57592.117,0.09290558,-9.0,-9.0,18.451132,18.540688,1.0,1.0,-9
,124363533,NULL,2,2.8645603811450875,1.0,-9,-9,0,101,23,141,163,0.02636083,0.0013180415,-9.0,-9.0,-9.0,-9.0,0.0,300.0,-9,-9.0,-9.0,0.0,0.0,-9.0,-9.0,-9.0,999.0,999.0,-9.0,-9.0,57458.082,-9,SALT2.WFIRST-H17,6,1,UNKNOWN,317,1,0,1,0.14827746,0.14720915,0.14827746,-87.79753,39.223415,-0.022519,2.86456036567688,0.026454661,57458.035,0.032284915,-9.0,-9.0,20.053347,20.10028,1.0,1.0,-9
,120475786,NULL,2,5.446362937234302,1.0,-9,-9,0,101,42,165,206,0.022678046,0.0011339023,-9.0,-9.0,-9.0,-9.0,0.0,300.0,-9,-9.0,-9.0,0.0,0.0,-9.0,-9.0,-9.0,999.0,999.0,-9.0,-9.0,57629.938,-9,SALT2.WFIRST-H17,6,1,UNKNOWN,338,1,0,1,0.29929033,0.29828295,0.29929033,-153.81381,40.942772,-0.0234933,5.4463629722595215,0.021109732,57632.477,0.0077110054,-9.0,-9.0,21.287857,21.209156,1.0,1.0,-9
,101712602,NULL,2,0.9519917603624842,1.0,-9,-9,0,101,53,208,260,0.029481145,0.0014740572,-9.0,-9.0,-9.0,-9.0,0.0,300.0,-9,-9.0,-9.0,0.0,0.0,-9.0,-9.0,-9.0,999.0,999.0,-9.0,-9.0,57679.04,-9,SALT2.WFIRST-H17,6,1,UNKNOWN,352,1,0,1,0.11761785,0.11576325,0.11761785,155.70459,38.659367,0.0020025002,0.9519917368888855,0.028507305,57679.4,0.1567895,-9.0,-9.0,19.27869,19.40735,1.0,1.0,-9
,11337994,NULL,2,3.2937010649608274,1.0,-9,-9,0,101,25,262,286,0.02496931,0.0012484655,-9.0,-9.0,-9.0,-9.0,0.0,300

In [ ]:

u, c = np.unique(test_ob.df_metadata['SNID'], return_counts=True)
dup = u[c > 1]

In [ ]:
print(dup)

In [ ]:
test_ob.df_data.write("/media/biswajit/drive/Kilonova_datasets/ZTF_20190512/test_master_PHOT.FITS", format='fits', overwrite=True)
test_ob.df_metadata.write("/media/biswajit/drive/Kilonova_datasets/ZTF_20190512/test_master_HEAD.FITS", format='fits', overwrite=True)

##### np.random.shuffle(data_ob2.df_data)

In [ ]:
train_ob = create_dataset(m_numbers, 21,40,4000,19000)

In [ ]:
train_ob.df_data.write("/media/biswajit/drive/Kilonova_datasets/ZTF_20190512/train_master_PHOT.FITS", format='fits', overwrite=True)
train_ob.df_metadata.write("/media/biswajit/drive/Kilonova_datasets/ZTF_20190512/train_master_HEAD.FITS", format='fits', overwrite=True)

In [ ]:
len(train_ob.df_metadata[train_ob.target_col_name])

In [ ]:
print(train_ob.df_data['SNID'])

In [ ]:
len(np.unique(train_ob.df_data['SNID']))

##### test_ob.df_data.write("/media/biswajit/drive/Kilonova_datasets/ZTF_20190512/test_PHOT.FITS", format='fits', overwrite=True)
test_ob.df_metadata.write("/media/biswajit/drive/Kilonova_datasets/ZTF_20190512/test_HEAD.FITS", format='fits', overwrite=True)

In [ ]:
len(data_ob2.df_metadata[data_ob2.target_col_name])